In [2]:
# Import required libraries:

import numpy as np
import pandas as pd
import os # this library provides easy access to the local file directories

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2 # opencv library for image processing purposes

from sklearn import model_selection # for train-test split

# Import the image augmentation library.
import imgaug as ia
import imgaug.augmenters as iaa

# Import the VGG-16 model:
from tensorflow.keras.applications.vgg16 import VGG16

# Import required Keras classes
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [16]:
# Specify some model parameters:

image_size = 128  # Each image in the dataset will be resized to (image_size x image_size x 3)
test_size  = 0.50 # Choose the ratio of test set
valid_portion = 0.20

data_augmentation = True # Choose whether to apply augmentation or not
ia.seed(1)

In [ ]:
def augmentation_function(X, y):
    # Save the previous versions of data. 
    X_temp = X
    y_temp = y
    
    # First augmentation: Apply horizontal mirroring on each image.
    seq = iaa.Sequential([
        iaa.Fliplr(1.0)
    ])
    X = np.concatenate((X, seq(images = X_temp)), axis=0)
    y = np.concatenate((y, y_temp), axis=0)
        
    # Second augmentation: Add Gaussian noise on each image. 
    # The noise on each pixel and channels is independently sampled from normal dist. of N(0, 0.25*255).
    seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(scale=(0, 0.25*255), per_channel=True)
    ])
    X = np.concatenate((X, seq(images = X_temp)), axis=0)
    y = np.concatenate((y, y_temp), axis=0)
      
    # Third augmentation: Random cropping. We randomly pick a window from each image.
    seq = iaa.Sequential([
        iaa.Crop(px=(0, 32)) # crop images from each side by 0 to 32px (randomly chosen)
    ])
    X = np.concatenate((X, seq(images = X_temp)), axis=0)
    y = np.concatenate((y, y_temp), axis=0)
        
    # Fourth augmentation: Rotate (between -45 to 45 degrees) and shear (from -16 to 16 degrees). 
    seq = iaa.Sequential([
        iaa.Affine(rotate=(-45, 45), shear=(-16, 16))
    ])
    X = np.concatenate((X, seq(images = X_temp)), axis=0)
    y = np.concatenate((y, y_temp), axis=0)
    
    # Fifth augmentation: Play with the colors.
    # First, convert the images to HSV and add random values (-50 to 50) to H-S channels of the image. Then turn back to RGB.
    # Second, change brightness by changing V (value) of HSV.
    seq = iaa.Sequential([
        iaa.AddToHueAndSaturation((-50, 50), per_channel=True),
        iaa.WithBrightnessChannels(iaa.Add((-50, 50)))
    ])
    X = np.concatenate((X, seq(images = X_temp)), axis=0)
    y = np.concatenate((y, y_temp), axis=0)
    
    del X_temp, y_temp, seq
    return X, y

In [ ]:
#RUN THIS BLOCK ONLY ONCE. IT WILL SAVE 2 NUMPY ARRAYS ON THE DIRECTORY, SO THAT YOU CAN LOAD THEM THEREAFTER. 

# folder should be the directory of the SKU_Recognition_Dataset folder. Change it accordingly. 
folder = 'C:/Users/Mecha/Desktop/Graduate_Courses/EE58J_DataMining/HW1/SKU_Recognition_Dataset'

# We will use 2 for loops in order to go through all confectionery (or icecream) SKUs (class labels) and images respectively.
# For each image, we will first read, then normalize its size, and stack them in X-y arrays to perform augmentation

y = [] # We will collect the labels of dataset in this list
X = np.empty((0, image_size, image_size, 3), dtype = np.uint8) # Each image will be stored in this 4D array
cls = 0
classNameDic = {}
classList = []

#category_name = 'icecream'
category_name = 'confectionery'

folder = folder+'/'+category_name   # folder is the directory of the given category folder

for class_name in os.listdir(folder): # .listdir() method willl return a list that contains all the folder names
    print(cls)
    
    classNameDic.update({cls: class_name})
    classList.append(class_name)
    class_folder = folder+'/'+class_name  # class_folder is the directory of a SKU folder that contains ~100 jpeg images
    for image_name in os.listdir(class_folder):
            
        # Create the direct path to a single image:
        final_directory = class_folder+'/'+image_name
        
        # Read the image as a (x,y,3) numpy array:
        image = cv2.imread(final_directory)
            
        # Resize it to (image_size,image_size,3) array:
        image = cv2.resize(image, (image_size,image_size))
        
        # Convert the BGR to HSV format (optional):
        # image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        
        # Store data in 4D numpy array
        X = np.concatenate((X, np.expand_dims(image, axis=0)), axis=0)
        y.append(cls)
        
    cls += 1
    
y = np.asarray(y)

# Save the data to local folder. You only run this block once. You can load the data once you save it.
np.save('X_icecream.npy', X)
np.save('y_icecream.npy', y)

In [ ]:
# RUN THIS BLOCK TO SEE THE EXAMPLES OF AUGMENTED IMAGES. THIS BLOCK DOES NOT HAVE ANY OTHER FUNCTION.

final_directory = "C:/Users/Mecha/Desktop/Graduate_Courses/EE58J_DataMining/HW1/SKU_Recognition_Dataset/icecream/sku.415/crop_1783040.jpg"

image_test = cv2.imread(final_directory)
image_test = cv2.resize(image_test, (image_size,image_size))
image_test = np.expand_dims(image_test, axis=0)

X = np.empty((0, image_size, image_size, 3), dtype = np.uint8) # Each image will be stored in this 4D array
X = np.concatenate((X, image_test), axis=0)

y = np.array([1])

# Apply data augmentation

X_aug, y_aug = augmentation_function(X, y)

print(X_aug.shape)
    
cv2.imshow("test", X_aug[5,:,:,:])
#cv2.imshow("test", image_test[0,:,:,:])
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('C:/Users/Mecha/Desktop/Graduate_Courses/EE58J_DataMining/HW4/5.jpg', X_aug[5,:,:,:])

In [18]:
# Load the data from local driver.
X = np.load('X_confec.npy')
y = np.load('y_confec.npy')

In [19]:
# Split the data into test and training data. By choosing stratify=y, each class will be split approximately by 20-80 ratio. 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                                                random_state = 2,
                                                stratify=y,
                                                test_size=test_size)
print(X_train.shape, y_train.shape)

# Apply data augmentation on the training set:
if data_augmentation == 1:
    X_train, y_train = augmentation_function(X_train, y_train)
print(X_train.shape, y_train.shape)

# Split the traning data into training and validation data. By choosing stratify=y, each class will be split with equal ratio. 
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X_train, y_train,
                                                random_state = 3,
                                                stratify=y_train,
                                                test_size=valid_portion)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

del X, y

(1032, 128, 128, 3) (1032,)
(6192, 128, 128, 3) (6192,)
(4953, 128, 128, 3) (4953,)
(1239, 128, 128, 3) (1239,)


In [20]:
# load the VGG16 model pretrained on ImageNet dataset. include_top=False discards the last fully connected layer.
# So, we can add any fully connected layer(s) at the end.
VGG16_model = VGG16(weights='imagenet', include_top=False, input_shape = (image_size, image_size, 3) )

# add a global spatial average pooling layer
out = VGG16_model.output
out = GlobalAveragePooling2D()(out)
# let's add two fully-connected layers:
#out = Dense(512, activation='relu')(out)
#out = Dropout(0.25)(out)
out = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(out)
out = Dropout(0.20)(out)
# and a logistic layer (output layer). Number of units will be equal to number of classes.
predictions = Dense(np.unique(y_train).size, activation='softmax')(out)

# this is the model we will train
model = Model(inputs=VGG16_model.input, outputs=predictions)

# Train only the top layers. i.e. freeze all convolutional layers
#for layer in VGG16_model.layers:
#    layer.trainable = False
    
for layer in model.layers[:17]:
    layer.trainable = False
for layer in model.layers[17:]:
    layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
# train the model on the new data for a few epochs
epochs = 30
batch_size = 64

callback = EarlyStopping(monitor='val_accuracy', patience=4)

model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, callbacks=[callback], validation_data = (X_valid, y_valid))

# Make predictions on test set
y_prediction = model.predict(x=X_test.astype("float16"), batch_size=batch_size, verbose=2)
y_prediction = np.argmax(y_prediction, axis=1)

# Find the accuracy score and print it:
score = 100 * sum(y_prediction == y_test) / y_prediction.shape[0]
print("Test set accuracy is {}".format(score))

# Find training set accuracy:
y_train_prediction = model.predict(x=X_train.astype("float16"), batch_size=batch_size, verbose=2)
y_train_prediction = np.argmax(y_train_prediction, axis=1)

# Find the accuracy score and print it:
score = 100 * sum(y_train_prediction == y_train) / y_train_prediction.shape[0]
print("Training set accuracy is {}".format(score))



Train on 4953 samples, validate on 1239 samples
Epoch 1/30
4953/4953 [==============================] - 1559s 315ms/sample - loss: 6.0169 - accuracy: 0.2863 - val_loss: 3.0310 - val_accuracy: 0.5246
Epoch 2/30
4953/4953 [==============================] - 1555s 314ms/sample - loss: 2.7702 - accuracy: 0.5389 - val_loss: 2.1935 - val_accuracy: 0.6651
Epoch 3/30
4953/4953 [==============================] - 1555s 314ms/sample - loss: 2.0375 - accuracy: 0.6638 - val_loss: 1.7530 - val_accuracy: 0.7264
Epoch 4/30
4953/4953 [==============================] - 1543s 312ms/sample - loss: 1.5925 - accuracy: 0.7444 - val_loss: 1.4025 - val_accuracy: 0.8241
Epoch 5/30
4953/4953 [==============================] - 1542s 311ms/sample - loss: 1.2295 - accuracy: 0.8157 - val_loss: 1.1963 - val_accuracy: 0.8216
Epoch 6/30
4953/4953 [==============================] - 1537s 310ms/sample - loss: 0.9561 - accuracy: 0.8801 - val_loss: 0.9794 - val_accuracy: 0.8660
Epoch 7/30
4953/4953 [========================

In [ ]:
del model, VGG16_model

In [ ]:
print(model.summary())